In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

We collect lexical co-occurrence statistics on all words in
the English Wikipedia, using the WikiExtractor tool2 to retrieve
plain text from the April 2015 dump (ca. 2.8B words),
and using simple regular expressions to segment sentences
and words, and remove URLs and punctuation. We perform
no POS tagging, lemmatisation, case normalisation,
or removal of numbers or symbols.

In [2]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On Simple English wiki

sf = open(datapath+'/Corpora/wiki/simple_20200601/simple_20200601_v2.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

April is the fourth month of the year, and comes between March and May. 

 It is one of four months to have 30 days. 

 April always begins on the same day of week as July, and additionally, January in leap years. 

 April always ends on the same day of the week as December. 

 April's flowers are the Sweet Pea and Daisy. 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

In [5]:
simp = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/','simple_20200601_v2.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

We collect word frequency information with the
SRILM language modelling toolkit (Stolcke, 2002), counting
n-grams (n <= 3), treating MWEs as contiguous bigrams
and trigrams), and identify MWE candidates by computing
the Poisson collocation measure (Quasthoff and Wolff,
2002) for all bigrams and trigrams (ca. 23M n-grams).
This method should be readily extensible to include longer
n-grams.

In [7]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [8]:
# Load n-gram list produced earlier

ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/ngram_eval.pkl')

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [9]:
# Stopwords from corpus - 50 most frequent
from nltk import FreqDist

fdist = FreqDist(word_gen(simp, sent_mark=''))

stop = set( word for word, f in fdist.most_common(50))

In [10]:
stop

{'0',
 '1',
 '2',
 '3',
 '4',
 'A',
 'American',
 'He',
 'In',
 'It',
 'References',
 'The',
 'This',
 'a',
 'also',
 'an',
 'and',
 'are',
 'as',
 'at',
 'be',
 'born',
 'by',
 'can',
 'first',
 'for',
 'from',
 'had',
 'has',
 'have',
 'he',
 'his',
 'in',
 'is',
 'it',
 'not',
 'of',
 'on',
 'one',
 'or',
 'people',
 'that',
 'the',
 'their',
 'they',
 'to',
 'was',
 'were',
 'which',
 'with'}

In [11]:
del fdist

In [12]:
min_freq = 20

In [13]:
ngram_eval

,ngram,freq,poisson,len,batch
0,"(Ving, Rhames)",20,-605.625590,2,1
1,"(Grădina, Zoologică)",20,-605.625590,2,1
2,"(Gharb-Chrarda-Beni, Hssen)",20,-605.625590,2,1
3,"(Karlovy, Vary)",20,-607.033377,2,1
4,"(waystations, shuku-eki)",20,-607.033377,2,1
...,...,...,...,...,...
149995,"(first, country, to)",45,-3071.582792,3,2
149996,"(from, A)",58,-3071.612088,2,-2
149997,"(1998, 8)",64,-3071.704678,2,6
149998,"(had, tested, positive)",55,-3071.728206,3,4


In [14]:
batch_count = max(ngram_eval.batch)

In [15]:
ngram_eval.batch.value_counts()

 1     45301
 2     33312
 3     24686
 4     16668
 5     10517
 6      7502
 7      4240
 8      2702
 9      1905
 10     1051
-2       832
 11      450
-1       300
 12      247
 13      123
 14      107
 15       57
Name: batch, dtype: int64

In [16]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [17]:
def mwe_score(exp, model, stats_frame):
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    oldn = []
    newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(1000)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in model.dictionary:

        mwv = model.word_vectors[model.dictionary[mwetoken]]

        for w in exp:
            if w in model.dictionary:
                cvs.append(model.word_vectors[model.dictionary[w]])

                #oldn.append(model.most_similar(w, number=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.word_vectors[model.dictionary[w]], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = model.most_similar(mwetoken, number=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [18]:

#batch_model = Glove.load(datapath+'/Models/2 GloVe/simple_batch{}.model'.format(bb+1))
batch_model = Glove.load(datapath+'/Models/2 GloVe//simp_glove_vocab_batch2.model')

batch_dict = batch_model.dictionary
batch_vect = batch_model.word_vectors

In [19]:
len(batch_dict)

333309

In [20]:
ngram_eval.iloc[2000]

ngram      (X-Men, Origins)
freq                     22
poisson             -832.45
len                       2
batch                     2
Name: 2000, dtype: object

In [21]:
len(batch_vect[batch_dict['X-Men+Origins']])

300

In [22]:
len(batch_vect[batch_dict['X-Men']])

300

In [23]:
len(batch_vect[batch_dict['Origins']])

300

In [24]:
cosim(batch_vect[batch_dict['X-Men']] , batch_vect[batch_dict['X-Men+Origins']] )

0.23099183684731814

In [25]:
cosim(batch_vect[batch_dict['Origins']] , batch_vect[batch_dict['X-Men+Origins']] )

-0.24027957618868892

In [26]:
batch_model.most_similar('X-Men+Origins')

[('Hōan', 0.9959869363058576),
 ('Coo', 0.9955508478483007),
 ('1931–2014', 0.9954400864683283),
 ('Continuous', 0.9949582512584049)]

In [31]:
batch_model.most_similar('X-Men')

[('TheXverse.com', 0.7612342016656892),
 ('Astonishing', 0.5977794198210621),
 ('Apocalypse', 0.5654931846543487),
 ('Marvel.com', 0.49236101200406673)]

In [32]:
batch_model.most_similar('Origins')

[('Sfida', 0.6174009132645951),
 ('Kaitos', 0.5746709058795924),
 ('The+origins', 0.5296782733795666),
 ('left:30', 0.5097731461144712)]

In [28]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading GloVe model')
    
    batch_model = Glove.load(datapath+'/Models/2 GloVe//simp_glove_vocab_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
     # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim'])  #, 'base_nearest', 'mwe_nearest'

    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
            
        statsf = mwe_score(exp,batch_model,statsf)

      #Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')    
    

Processing batch 1 of 15
Loading GloVe model
Gathering MWE stats
 MWE 0/45301: Ving+Rhames
 MWE 1000/45301: Real+Murcia
 MWE 2000/45301: Nguyen+dynasty's
 MWE 3000/45301: Cyndi+Lauper
 MWE 4000/45301: Dragon+Tattoo
 MWE 5000/45301: Southern+Ontario
 MWE 6000/45301: will+decide
 MWE 7000/45301: been+affected
 MWE 8000/45301: and+Gone
 MWE 9000/45301: being+chased
 MWE 10000/45301: decoration+presented
 MWE 11000/45301: Swiss+Federal+Councillor
 MWE 12000/45301: was+welcomed
 MWE 13000/45301: at+Duke
 MWE 14000/45301: more+copies
 MWE 15000/45301: Without+Borders
 MWE 16000/45301: insurance+companies
 MWE 17000/45301: from+disease
 MWE 18000/45301: the+Willis+Tower
 MWE 19000/45301: mounted+in
 MWE 20000/45301: his+twin+brother
 MWE 21000/45301: his+queen
 MWE 22000/45301: Nazi+Germany+in
 MWE 23000/45301: Manchester+in
 MWE 24000/45301: now+used+by
 MWE 25000/45301: as+the+Kingdom
 MWE 26000/45301: Consadole+Sapporo+Football
 MWE 27000/45301: was+eight+years
 MWE 28000/45301: to+his+wor

In [ ]:
# Removed
    #print('Gathering MWE stats')
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    #statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
    #                               'cosine_sim', 'base_nearest', 'mwe_nearest'])

    #for exp in batch_dfs[bb].ngram:
    #    statsf = mwe_score(exp,batch_model,statsf)

    #  Join back onto DataFrame
    #batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')

In [29]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/2 GloVe/Results/simple_vocab_output_001.csv', index=False)

In [30]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(the, highest-rated)",22,-1025.646943,2,1,"[1, 0]","[-1.9905872957151742, -1.9976834386702171, -1....","[[0.006619947186335439, 0.14868128607595, -0.1...","[nan, -0.9999907006105433]",-0.999991
1,"(was, facelifted, in)",24,-1540.866708,3,5,"[1, 0, 1]","[-5.886014101390828, -5.921196131138483, 5.922...","[[0.16271233754597866, -0.04303778304083927, 0...","[nan, -0.9999871461769124, nan]",-0.999987
2,"(the, station's)",44,-2069.249614,2,1,"[1, 0]","[1.8662915200814416, 1.8526063958224677, 1.870...","[[0.006619947186335439, 0.14868128607595, -0.1...","[nan, -0.9999648035199541]",-0.999965
3,"(in, the, shade)",21,-1431.207176,3,2,"[1, 1, 0]","[-1.8775673300712086, 1.8953319133441393, -1.8...","[[0.10856749640439656, -0.32831220974975284, 0...","[nan, nan, -0.9999537329974788]",-0.999954
4,"(The, Comunità)",36,-1639.428758,2,1,"[1, 0]","[1.426591690916393, 1.4026078498029213, 1.4056...","[[-0.00473311056862602, 0.2508571552947894, -0...","[nan, -0.9999526702180579]",-0.999953
...,...,...,...,...,...,...,...,...,...,...
149995,"(in, its, 1)",43,-3066.527125,3,-1,NaN,NaN,NaN,NaN,NaN
149996,"(the, disease, at)",44,-3070.154899,3,-1,NaN,NaN,NaN,NaN,NaN
149997,"(America, during, the)",45,-3071.046482,3,-1,NaN,NaN,NaN,NaN,NaN
149998,"(4, 3, 4)",46,-3071.054761,3,-2,NaN,NaN,NaN,NaN,NaN


In [31]:
all_batches_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [32]:
all_batches_light.to_csv(datapath+'/Models/2 GloVe/Results/simple_vocab_light_001.csv', index=False)